## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-01-12-53 +0000,bbc,Franklin the Turtle and Sabrina Carpenter in t...,https://www.bbc.com/news/articles/cy7v022n5kyo...
1,2025-12-03-01-11-00 +0000,wsj,Marathon Russia-U.S. Meeting Yields No Ukraine...,https://www.wsj.com/world/russia/trump-envoys-...
2,2025-12-03-01-09-28 +0000,nyt,University of Oklahoma Instructor on Leave Aft...,https://www.nytimes.com/2025/12/02/us/oklahoma...
3,2025-12-03-01-06-59 +0000,nyt,Trump Calls Affordability a ‘Con Job’ as His E...,https://www.nytimes.com/2025/12/02/us/politics...
4,2025-12-03-01-05-14 +0000,nyt,Trump Calls Somalis ‘Garbage’ He Doesn’t Want ...,https://www.nytimes.com/2025/12/02/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,65
298,new,21
51,up,18
32,he,15
44,drug,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,209
337,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,134
349,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...,130
78,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...,121
236,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...,121


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,209,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
163,74,2025-12-02-19-29-00 +0000,wsj,Defense Secretary Pete Hegseth said he didn’t ...,https://www.wsj.com/politics/national-security...
349,58,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...
78,56,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...
337,46,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
210,43,2025-12-02-16-30-32 +0000,wapo,"Pope Leo, leaving Beirut, calls for peace in M...",https://www.washingtonpost.com/world/2025/12/0...
212,41,2025-12-02-16-04-52 +0000,nypost,Samsung unveils first triple-folding phone ahe...,https://nypost.com/2025/12/02/business/samsung...
107,39,2025-12-02-21-50-00 +0000,nypost,"Mexican migrant accused of killing boy, 11, in...",https://nypost.com/2025/12/02/us-news/mexican-...
105,39,2025-12-02-21-52-49 +0000,nyt,Michael and Susan Dell Pledge $6 Billion in In...,https://www.nytimes.com/2025/12/02/business/de...
18,38,2025-12-03-00-33-35 +0000,latimes,L.A. City Council votes against proposal to ba...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
